In [1]:
!pip install tensorflow

In [5]:
! pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.5/124.9 MB 4.2 MB/s eta 0:00:30
    --------------------------------------- 1.6/124.9 MB 3.2 MB/s eta 0:00:39
    --------------------------------------- 2.1/124.9 MB 2.8 MB/s eta 0:00:44
    --------------------------------------- 2.9/124.9 MB 3.0 MB/s eta 0:00:41
   - -------------------------------------- 3.4/124.9 MB 2.9 MB/s eta 0:00:42
   - -------------------------------------- 3.9/124.9 MB 3.0 MB/s eta 0:00:41
   - -------------------------------------- 3.9/124.9 MB 3.0 MB/s eta 0:00:41
   - -------------------------------------- 4.7/124.9 MB 2.8 MB/s eta 0:00:43
   - -------------------------------------- 5.0/124.9 MB 2.7 MB/s eta 0:00:45
   - -------------------------------------- 5.5/124.9 MB 2.5 MB/s eta 0:00:49
   - -------------------------------------- 5.8/124.9 MB 2.5 MB/s eta 0:00:49


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
import xgboost as xgb
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [8]:
train_dir = r'C:\Users\shams\Desktop\Sections\Computer vision\Alzheimer_s Dataset\Alzheimer_s Dataset\train'
test_dir = r'C:\Users\shams\Desktop\Sections\Computer vision\Alzheimer_s Dataset\Alzheimer_s Dataset\test'

In [9]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [10]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [15]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 4220 images belonging to 4 classes.


In [17]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1279 images belonging to 4 classes.


In [19]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\shams\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features
train_labels = train_generator.classes

C:\Users\shams\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


132/132 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step


In [23]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step


In [25]:
# Use XGBoost 
xgb_classifier = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.3,
    max_depth=6,
    eval_metric='mlogloss')
xgb_classifier.fit(train_features, train_labels)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.3, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [27]:
# Make predictions on the test data
test_predictions = xgb_classifier.predict(test_features)

In [29]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 57.00%


In [31]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor9.h5')

In [33]:
joblib.dump(xgb_classifier, 'xgb_classifier.pkl')

['xgb_classifier.pkl']